In [1]:
import pandas as pd
import numpy as np
import pickle
#
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [2]:
#data
dataset = pd.read_csv('/home/kate/data/ClaimPrediction/fdata_v1_encd.csv', index_col=None)
target_column = 'hasclaim'

In [3]:
featureset=[
'acci_last_infractionage',
'carpoolind_encd',
'classcd_encd',
'driverage',
'drivernumber',
'estimatedannualdistance',
'gooddriverind_encd',
'maritalstatuscd_encd',
'mvrstatus_encd',
'mvrstatusage',
'ratingvalue',
'vehbodytypecd_encd',
'vehicleage',
'vehnumber',
'licenseage',
'gendercd_encd'
]
#add calculated column
dataset['licenseage']=dataset['driverage']-dataset['havelicenseage']
need_refline_set=['driverage', 
                  'drivernumber', 
                  'estimatedannualdistance', 
                  'ratingvalue',
                  'vehicleage',
                  'vehnumber',
                  'licenseage']

In [4]:
#models files dir
ModelsDir='/home/kate/Models/XGB/'

In [5]:
import glob
ModelsList=glob.glob('%s*.model_licenseage_gender'%ModelsDir)

In [6]:
#models images files dir
ModelsImgDir='/home/kate/Models/XGB/img/'

In [7]:
#models tables files dir
ModelsTblDir='/home/kate/Models/XGB/tbl/'

In [8]:
#xgb library
import xgboost as xgb

In [9]:
#Evaluation metric to be used in tuning
from sklearn.metrics import roc_auc_score,confusion_matrix
def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)
def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

In [10]:
# from https://xiaoxiaowang87.github.io/monotonicity_constraint/
def partial_dependency(model, X,  feature):

    """
    Calculate the dependency (or partial dependency) of a response variable on a predictor (or multiple predictors)
    1. Sample a grid of values of a predictor for numeric continuous or all unique values for categorical or discrete continuous.
    2. For each value, replace every row of that predictor with this value, calculate the average prediction.
    """

    X_temp = X.copy()
    
    if feature in ['estimatedannualdistance','ratingvalue']:
        # continuous
        grid = np.linspace(np.percentile(X_temp[feature], 0.1),
                       np.percentile(X_temp[feature], 99.5),
                       50)
    else:
        #categorical
        grid = X_temp[feature].unique()

    y_pred = np.zeros(len(grid))

    for i, val in enumerate(grid):
        X_temp[feature] = val
        d_temp=xgb.DMatrix(X_temp.values)
        y_pred[i] = np.average(model.predict(d_temp))

    return grid, y_pred

In [11]:
#splitting to train/test 
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(dataset.loc[:,featureset], dataset[target_column], test_size=0.2, random_state=42)

In [12]:
#prediction dataframes
y_pred_test=pd.DataFrame(index=y_test.index)
y_pred_test[target_column]=0
kfold = 10
for xgb_model_file in ModelsList:
    print(xgb_model_file)
    ModelName='Model '+xgb_model_file[24:25]
    #load saved model
    xgb_model = pickle.load(open(xgb_model_file, 'rb'))
    #prediction
    d_test=xgb.DMatrix(X_test.values)
    y_pred_test[target_column] +=  xgb_model.predict(d_test, ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    #feature importance
    feat_imp = pd.Series(xgb_model.get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances in  %s'%ModelName)
    #plt.ylabel('Feature Importance Score')
    #plt.show()
    #plt.savefig(ModelsImgDir+'FE %s.png'%ModelName)   
    feat_imp.to_csv(ModelsTblDir+'GL Feature Importance %s.csv'%ModelName)
    #partial dependency
    for f in featureset:
        if f in need_refline_set:
            continue
        print(f)
        #model partial dependency mpd dataframe
        grid, y_pred = partial_dependency(xgb_model,X,f)
        mpd=pd.concat([pd.Series(grid), pd.Series(y_pred)], axis=1)
        mpd.columns=[f,'pd']

        #real values binned for continuous or as is for categorical
        if f in ['estimatedannualdistance','ratingvalue']:
            rv=pd.concat([pd.Series(X[f]), pd.Series(y)], axis=1)
            feature_bucket_array = np.linspace(np.percentile(X[f], 0.1),
                       np.percentile(X[f], 99.5),
                       50)
            feature_bucket_array=np.insert(feature_bucket_array, 0, (X[f].min()-X[f].max()/2))
            feature_bucket_array=np.insert(feature_bucket_array, len(feature_bucket_array), (X[f].max()+X[f].max()/2))
            dummy=pd.DataFrame(feature_bucket_array)
            dummy.columns=[f]
            rv['bin'] = pd.cut(rv[f], feature_bucket_array)
            rv_grp=pd.concat([rv.groupby('bin')['hasclaim'].sum(),rv.groupby('bin')[f].count()], axis=1)
            rv_grp.columns=['hasclaim','count']
            rv_grp.reset_index(inplace=True)
            rv_grp=pd.concat([rv_grp,dummy], axis=1)
            rv_grp.dropna(inplace=True)
        else:
            rv=pd.concat([pd.Series(X[f]), pd.Series(y)], axis=1)
            rv_grp=pd.concat([rv.groupby(f)['hasclaim'].sum(),rv.groupby(f)[f].count()], axis=1)
            rv_grp.columns=['hasclaim','count']
            rv_grp.reset_index(inplace=True)

        if '_encd' in f:
            mpd_grp=pd.merge(mpd, rv_grp, on=f)
            
            #codes dictionary
            f_dict=dataset[[f.replace('_encd',''),f]].set_index(f).to_dict()
            f_dict_df=pd.DataFrame.from_dict(f_dict).reset_index()
            f_dict_df.columns=[f,f.replace('_encd','')]
            
            pd.merge(mpd_grp, f_dict_df, on=f).to_csv(ModelsTblDir+'GL PD %s %s.csv'%(ModelName,f))
        else:
            pd.merge(mpd, rv_grp, on=f).to_csv(ModelsTblDir+'GL PD %s %s.csv'%(ModelName,f))

        #fig, ax = plt.subplots()
        #fig.set_size_inches(8, 8)
        #plt.subplots_adjust(left = 0.17, right = 0.94, bottom = 0.15, top = 0.9)

        #ax.plot(grid, y_pred, '-', color = 'red', linewidth = 2.5, label='fit')
        #ax.plot(X[f], y, 'o', color = 'grey', alpha = 0.01)

        #ax.set_xlim(min(grid), max(grid))
        #ax.set_ylim(0.95 * min(y_pred), 1.05 * max(y_pred))

        #ax.set_xlabel(f, fontsize = 24)
        #ax.set_ylabel('Partial Dependence', fontsize = 24)
        #ax.set_title(ModelName, fontsize = 24)

        #plt.xticks(fontsize=20)
        #plt.yticks(fontsize=20)

        #plt.show()
        #plt.savefig(ModelsImgDir+'PD %s %s.png'%(ModelName,f))

/home/kate/Models/XGB/m_9.model_licenseage_gender
acci_last_infractionage
carpoolind_encd
classcd_encd
gooddriverind_encd
maritalstatuscd_encd
mvrstatus_encd
mvrstatusage
vehbodytypecd_encd
gendercd_encd
/home/kate/Models/XGB/m_5.model_licenseage_gender
acci_last_infractionage
carpoolind_encd
classcd_encd
gooddriverind_encd
maritalstatuscd_encd
mvrstatus_encd
mvrstatusage
vehbodytypecd_encd
gendercd_encd
/home/kate/Models/XGB/m_0.model_licenseage_gender
acci_last_infractionage
carpoolind_encd
classcd_encd
gooddriverind_encd
maritalstatuscd_encd
mvrstatus_encd
mvrstatusage
vehbodytypecd_encd
gendercd_encd
/home/kate/Models/XGB/m_2.model_licenseage_gender
acci_last_infractionage
carpoolind_encd
classcd_encd
gooddriverind_encd
maritalstatuscd_encd
mvrstatus_encd
mvrstatusage
vehbodytypecd_encd
gendercd_encd
/home/kate/Models/XGB/m_3.model_licenseage_gender
acci_last_infractionage
carpoolind_encd
classcd_encd
gooddriverind_encd
maritalstatuscd_encd
mvrstatus_encd
mvrstatusage
vehbodytypecd

In [13]:
#Prediction results
g=gini(y_test,y_pred_test)/gini(y_test,y_test)
print('Test Gini - %f'%g)

ROC_AUC=roc_auc_score(y_test, y_pred_test)
print('Test ROC_AUC - %f'%ROC_AUC)

#mean prediction value to convert to binary
m=y_pred_test.mean()

y_pred_test[y_pred_test >= m] = 1
y_pred_test[y_pred_test < m] = 0

print ('\nConfusion matrix\n')    
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
print('TP=%d FP=%d'%(tp,fp))
print('FN=%d TN=%d'%(fn,tn))

Test Gini - 0.420893
Test ROC_AUC - 0.710447

Confusion matrix

TP=930 FP=9727
FN=420 TN=15498
